# Lab 12 - Pivoting and unpivoting

This lab focuses on data transformation using pivoting and unpivoting operations with the polars library.

## Load your packages

You will need the `polars` package for this assignment. We'll also use `numpy` for some calculations.

In [5]:
# Import required packages
import polars as pl
from google.colab import files

uploaded = files.upload()

# Load datasets from uploaded files
billboard = pl.read_csv("billboard (2).csv")
economics_long = pl.read_csv("economics_long (2).csv", null_values=["NA", ""])
table2 = pl.read_csv("table2 (2).csv", null_values=["NA", ""])
us_rent_income = pl.read_csv("us_rent_income (2).csv", null_values=["NA", ""])
world_bank_pop = pl.read_csv("world_bank_pop (2).csv", null_values=["NA",""])

Saving billboard.csv to billboard (3).csv
Saving economics_long.csv to economics_long (3).csv
Saving table2.csv to table2 (3).csv
Saving us_rent_income.csv to us_rent_income (3).csv
Saving world_bank_pop.csv to world_bank_pop (3).csv


# Question 1: Pivot US Economic Indicator Data

## 1a. Pivot data

Load the `economics_long.csv` dataset from the `data/` folder. You'll notice that in the `economics_long` dataset's `value` column, there are different types of measurements. Transform this dataset so that the unique values for the `variable` field become columns of their own.

**Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.**

In [6]:
# Load economics_long data
import polars as pl

# Pivot data: variable values become columns, values fill them
economics_wide = economics_long.pivot(
    index=["date"],
    columns="variable",
    values="value",
    aggregate_function="first"
)

print("Economics wide dataset shape:", economics_wide.shape)
print(economics_wide.head(10))

Economics wide dataset shape: (574, 6)
shape: (10, 6)
┌────────────┬───────┬──────────┬─────────┬─────────┬──────────┐
│ date       ┆ pce   ┆ pop      ┆ psavert ┆ uempmed ┆ unemploy │
│ ---        ┆ ---   ┆ ---      ┆ ---     ┆ ---     ┆ ---      │
│ str        ┆ f64   ┆ f64      ┆ f64     ┆ f64     ┆ f64      │
╞════════════╪═══════╪══════════╪═════════╪═════════╪══════════╡
│ 1967-07-01 ┆ 506.7 ┆ 198712.0 ┆ 12.6    ┆ 4.5     ┆ 2944.0   │
│ 1967-08-01 ┆ 509.8 ┆ 198911.0 ┆ 12.6    ┆ 4.7     ┆ 2945.0   │
│ 1967-09-01 ┆ 515.6 ┆ 199113.0 ┆ 11.9    ┆ 4.6     ┆ 2958.0   │
│ 1967-10-01 ┆ 512.2 ┆ 199311.0 ┆ 12.9    ┆ 4.9     ┆ 3143.0   │
│ 1967-11-01 ┆ 517.4 ┆ 199498.0 ┆ 12.8    ┆ 4.7     ┆ 3066.0   │
│ 1967-12-01 ┆ 525.1 ┆ 199657.0 ┆ 11.8    ┆ 4.8     ┆ 3018.0   │
│ 1968-01-01 ┆ 530.9 ┆ 199808.0 ┆ 11.7    ┆ 5.1     ┆ 2878.0   │
│ 1968-02-01 ┆ 533.6 ┆ 199920.0 ┆ 12.3    ┆ 4.5     ┆ 3001.0   │
│ 1968-03-01 ┆ 544.3 ┆ 200056.0 ┆ 11.7    ┆ 4.1     ┆ 2877.0   │
│ 1968-04-01 ┆ 544.0 ┆ 200208.0 ┆ 12

/tmp/ipython-input-1530398975.py:5: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  economics_wide = economics_long.pivot(


## 1b. Calculate median unemployment for 2010-01-01 and beyond

Next, compute the median unemployment metric (as defined by `unemploy`) for 2010-01-01 and beyond.

**Please write code below. The output should show the median value. You must use the dataframe created in 1a.**

In [11]:
# YOUR CODE HERE: Filter for dates >= '2010-01-01' and calculate median unemployment
# Hint: You'll need to filter the dataframe and then calculate the median of the 'unemploy' column

economics_wide = economics_long.pivot(
    index=["date"],
    columns="variable",
    values="value",
    aggregate_function="first"
)

economics_wide = economics_wide.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d")
)

# Filter for 2010+, compute median unemployment
median_unemploy = (
    economics_wide
    .filter(pl.col("date") >= pl.lit("2010-01-01").str.strptime(pl.Date, "%Y-%m-%d"))
    .select(pl.col("unemploy").median())
)

median_value = median_unemploy[0, 0]

print(f"People unemployed: {median_value:,.0f}")

People unemployed: 12,471


/tmp/ipython-input-969471969.py:4: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  economics_wide = economics_long.pivot(


### Answer: The median unemployment for 2010-01-01 and beyond is: ____12,471 people______

# Question 2: Air Passenger data

## 2a. Unpivot data

The `AirPassengers` dataset is wide, which makes it difficult to aggregate. Transform the `AirPassengers` dataset from wide to long. The resulting dataset should have three columns: one column representing year, one column representing month, and one column representing the number of air passengers.

**Please write code below to unpivot the data. The output should be a DataFrame. You must "unpivot" the data to receive full credit on this question.**

In [17]:
import polars as pl
from google.colab import files

# Unpivot wide → long
air_long = air_passengers.unpivot(
    index=["Year"], # Use 'Year' with a capital 'Y'
    on=[c for c in air_passengers.columns if c != "Year"], # Use 'Year' here too
    variable_name="month",
    value_name="passengers"
)

# Preview
print(air_long.head(12))

shape: (12, 3)
┌──────┬───────┬────────────┐
│ Year ┆ month ┆ passengers │
│ ---  ┆ ---   ┆ ---        │
│ i64  ┆ str   ┆ i64        │
╞══════╪═══════╪════════════╡
│ 1949 ┆ Jan   ┆ 112        │
│ 1950 ┆ Jan   ┆ 115        │
│ 1951 ┆ Jan   ┆ 145        │
│ 1952 ┆ Jan   ┆ 171        │
│ 1953 ┆ Jan   ┆ 196        │
│ …    ┆ …     ┆ …          │
│ 1956 ┆ Jan   ┆ 284        │
│ 1957 ┆ Jan   ┆ 315        │
│ 1958 ┆ Jan   ┆ 340        │
│ 1959 ┆ Jan   ┆ 360        │
│ 1960 ┆ Jan   ┆ 417        │
└──────┴───────┴────────────┘


## 2b. Find the standard deviation between 1955 and 1960

Next, compute the standard deviation of passengers between (and including) the years 1955 and 1960.

**Please write code below. The output should show the standard deviation. You must use the dataframe created in 2a.**

In [20]:
# YOUR CODE HERE: Filter for years 1955-1960 and calculate standard deviation
# Hint: Filter the long format data for years between 1955 and 1960 (inclusive)
# Then calculate the standard deviation of the passenger values


std_passengers = (
    air_long
    .filter((pl.col("Year") >= 1955) & (pl.col("Year") <= 1960))
    .select(pl.col("passengers").std().alias("std_passengers"))
)


std_value = std_passengers[0, 0]

print(f"Standard Deviation (1955–1960): {std_value:,.2f}")
print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_value:.2f}")

Standard Deviation (1955–1960): 86.44
The standard deviation of airline passengers between 1955 and 1960 is: 86.44


### The standard deviation of airline passengers between (and including) the years 1955 and 1960 is: __________

# Question 3: WHO tuberculosis case data

## 3a. Pivot data

Load the `table2.csv` dataset from the `data/` folder. This contains data for country populations and tuberculosis case metrics. You'll notice that the metrics for `cases` and `population` are in the same column (`count`). Pivot the data so that the metrics for `cases` and `population` are in their own columns.

**Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.**

In [23]:
import polars as pl



# Pivot so 'type' values become separate columns
table2_wide = table2.pivot(
    index=["country", "year"],
    columns="type",
    values="count",
    aggregate_function="first"
)

print(table2_wide.head(10))

shape: (6, 4)
┌─────────────┬──────┬────────┬────────────┐
│ country     ┆ year ┆ cases  ┆ population │
│ ---         ┆ ---  ┆ ---    ┆ ---        │
│ str         ┆ i64  ┆ i64    ┆ i64        │
╞═════════════╪══════╪════════╪════════════╡
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   │
│ Afghanistan ┆ 2000 ┆ 2666   ┆ 20595360   │
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  │
│ Brazil      ┆ 2000 ┆ 80488  ┆ 174504898  │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 │
│ China       ┆ 2000 ┆ 213766 ┆ 1280428583 │
└─────────────┴──────┴────────┴────────────┘


/tmp/ipython-input-1220276543.py:6: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  table2_wide = table2.pivot(


## 3b. Calculate highest cases/population ratio for 1999

Now that you have `cases` and `population` in their own columns, you can perform analysis. Create a new column in this dataset called `ratio` that divides `cases` by `population`. Next, filter the data to only include rows from the `year` of 1999. What is the country with the highest `cases`/`population` ratio?

**Please write code below. You must use the dataframe created in 3a.**

In [27]:
# Add ratio column (cases per 100,000 people)
table2_ratio = table2_wide.with_columns(
    ((pl.col("cases") / pl.col("population")) * 100_000).alias("ratio")
)

# Filter for 1999 and sort descending by ratio
highest_ratio_1999 = (
    table2_ratio
    .filter(pl.col("year") == 1999)
    .sort("ratio", descending=True)
    .select(["country", "ratio"])
    .head(1)
)

# Extract values
country = highest_ratio_1999[0, 0]
ratio_val = highest_ratio_1999[0, 1]

print(f"Highest cases per 100,000 people in 1999: {country} ({ratio_val:,.2f})")

Highest cases per 100,000 people in 1999: Brazil (21.94)


### Answer: The country with the highest cases/population ratio in 1999 is: BRAZIL__________

## Submission Instructions

1. Complete all code cells above
2. Run all cells to ensure they execute without errors
3. Fill in the answer blanks with your computed values
4. Save and submit your completed notebook